In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
data = pd.read_excel('Input.xlsx')
stop_auditor = pd.read_csv('StopWords_Auditor.txt', names=[0])
stop_currency = pd.read_csv('StopWords_Currencies.txt', delimiter='|', encoding='latin-1', names = ['currency','country'])
stop_date_nums = pd.read_csv('StopWords_DatesandNumbers.txt', names=[0])
stop_general = pd.read_csv('StopWords_Generic.txt', names=[0])
stop_generic = pd.read_csv('StopWords_GenericLong.txt', names=[0])
stop_geo = pd.read_csv('StopWords_Geographic.txt', names=[0])
stop_names = pd.read_csv('StopWords_Names.txt', names=[0])

In [3]:
data

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [4]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage = requests.get(data['URL'][1], headers=headers).text
soup = BeautifulSoup(webpage, 'lxml')

In [5]:
words = []
for i in soup.find_all('p'):
    words.append((i.text.split()))
for i in soup.find_all('h1', class_ = 'entry-title'):
    words.append(i.text.strip())

new_words = []
for i in range(len(words)):
    for j in words[i]:
        new_words.append(j)
raw_words = new_words            

In [6]:
stopwords_list = (list(stop_auditor.values) + list(stop_currency.values) + list(stop_date_nums.values) + list(stop_general.values) + list(stop_generic.values) + list(stop_geo.values) + list(stop_names.values))
stopwords = []
for i in range(len(stopwords_list)):
    stopwords.append(stopwords_list[i][0])
new_words = ([word for word in new_words if word not in stopwords])    

In [7]:
positive = pd.read_csv('positive-words.txt', encoding='latin-1', names = [0]).values
negative = pd.read_csv('negative-words.txt', encoding='latin-1', names = [0]).values
p_words = []
n_words = []
for i in range(len(positive)):
    p_words.append(positive[i][0])
    
for i in range(len(negative)):
    n_words.append(negative[i][0]) 
positive_words = ([word for word in p_words if word not in stopwords])
negative_words = ([word for word in n_words if word not in stopwords])    

In [8]:
new_words = ' '.join(new_words)
token_words = word_tokenize(new_words)

In [42]:
def sentiment_analysis (token_words):
    count_positive = 0
    count_negative = 0
    for word in token_words:
        if word in positive_words:
            count_positive += 1
        if word in negative_words:
            count_negative += 1
    subjectivity = ((count_positive + count_negative)/len(new_words)) + 0.000001
    if (count_negative + count_positive) > 0 :
        polarity = (count_positive - count_negative)/(count_negative + count_positive) + 0.000001  
    else:
        polarity = np.nan
    return count_positive, count_negative, subjectivity, polarity

In [10]:
sentiment_analysis(token_words)

(56, 38, 0.01734097417450655, 0.19149036170212766)

In [11]:
def syllable_count(word):
    word = word.lower()
    syllables = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        syllables += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            syllables += 1
    if word.endswith("e"):
        syllables -= 1
    if syllables == 0:
        syllables += 1
    return syllables

In [14]:
def total_syllables(token_words):
    number_of_syllables = 0
    for i in range(len(token_words)):
        syllables = syllable_count(token_words[i])
        number_of_syllables += syllables
    return number_of_syllables    

In [15]:
total_syllables(token_words)

1686

In [35]:
def complex_word_count(token_words):
    complex_words = 0
    for i in range(len(token_words)):
        syllables = syllable_count(token_words[i])
        if syllables > 2 :
            complex_words +=1
    return complex_words        

In [36]:
complex_word_count(token_words)

455

In [24]:
def analysis_of_readibility(new_words):
    token_sentence = sent_tokenize(new_words)
    
    complex_words = 0
    for i in range(len(token_words)):
        syllables = syllable_count(token_words[i])
        if syllables > 2 :
            complex_words +=1
            
    percent_complex_words = complex_words/len(token_words)       
    avg_sentence_length = len(token_words)/len(token_sentence)
    fog_index = (percent_complex_words + avg_sentence_length) * 0.4
    return percent_complex_words, avg_sentence_length, fog_index
    
    

In [25]:
analysis_of_readibility(new_words)

(0.2470076169749728, 11.4875, 4.69380304678999)

In [26]:
def avg_words_per_sentence(token_words, token_sentence):
    return (len(token_words)/len(token_sentence))

In [27]:
token_sentence = sent_tokenize(new_words)
avg_words_per_sentence(token_words, token_sentence)

11.4875

In [28]:
def word_count(new_words):
    nltk_stopwords = (nltk.corpus.stopwords.words('english'))
    clean_words = ([word for word in new_words.split() if word not in nltk_stopwords])
    clean_words = ' '.join(clean_words)
    clean_words = [char for char in clean_words if char not in string.punctuation]
    clean_words = ''.join(clean_words)
    total_cleaned_words = len(clean_words.split())
    return total_cleaned_words
    
    

In [29]:
word_count(new_words)

688

In [30]:
def personal_pronouns(raw_words):
    personal_pronouns = ['I', 'we', 'my', 'ours', 'us', 'We', 'My', 'Ours', 'Us']
    pronouns = []
    for i in range(len(raw_words)):
        if raw_words[i] in personal_pronouns:
            pronouns.append(raw_words[i]) 
    return (len(pronouns))        
    

In [31]:
personal_pronouns(raw_words)

6

In [32]:
def avg_word_length(clean_words):
    word_length_per_word = []
    for i in range(len(clean_words.split())):
        word = clean_words.split()[i]
        word_length = 0
        for j in range(len(word)):
            word_length = word_length + 1
        word_length_per_word.append(word_length)    
    avg_word_length = sum(word_length_per_word)/len(clean_words.split())  
    return avg_word_length

In [33]:
    nltk_stopwords = (nltk.corpus.stopwords.words('english'))
    clean_words = ([word for word in new_words.split() if word not in nltk_stopwords])
    clean_words = ' '.join(clean_words)
    clean_words = [char for char in clean_words if char not in string.punctuation]
    clean_words = ''.join(clean_words)
    avg_word_length(clean_words)

6.5523255813953485

In [43]:
positive_score_ = []
negative_score_ = []
polarity_score_ = []
subjectivity_score_ = []
percent_complex_words_ = []
avg_sentence_length_ = []
fog_index_ = []
avg_words_per_sentence_ = []
complex_word_count_ = []
word_count_ = []
syllable_count_ = []
personal_pronouns_ = []
avg_word_length_ = []

for i in range(len(data['URL'])):
    print(i)
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    webpage = requests.get(data['URL'][i], headers=headers).text
    soup = BeautifulSoup(webpage, 'lxml')
    words = []
    for i in soup.find_all('p'):
        words.append((i.text.split()))
    for i in soup.find_all('h1', class_ = 'entry-title'):
        words.append(i.text.strip())

    new_words = []
    for i in range(len(words)):
        for j in words[i]:
            new_words.append(j)
    raw_words = new_words  
    new_words = ' '.join(new_words)
    token_words = word_tokenize(new_words)
    token_sentence = sent_tokenize(new_words)

    nltk_stopwords = (nltk.corpus.stopwords.words('english'))
    clean_words = ([word for word in new_words.split() if word not in nltk_stopwords])
    clean_words = ' '.join(clean_words)
    clean_words = [char for char in clean_words if char not in string.punctuation]
    clean_words = ''.join(clean_words)
    
    if len(new_words) != 0:
        positive, negative, polarity, subjectivity = sentiment_analysis(token_words)
        positive_score_.append(positive)
        negative_score_.append(negative)
        polarity_score_.append(polarity)
        subjectivity_score_.append(subjectivity)

        complex_words, sentence_length, index = analysis_of_readibility(new_words)
        percent_complex_words_.append(complex_words)
        avg_sentence_length_.append(sentence_length)
        fog_index_.append(index)

        words_per_sentence = avg_words_per_sentence(token_words, token_sentence)
        avg_words_per_sentence_.append(words_per_sentence)

        complex_words = complex_word_count(new_words)
        complex_word_count_.append(complex_words)

        word_counts = word_count(new_words)
        word_count_.append(word_counts)

        number_of_syllables = total_syllables(token_words)
        syllable_count_.append(number_of_syllables)

        pronoun_count = personal_pronouns(raw_words)
        personal_pronouns_.append(pronoun_count)

        word_lnth = avg_word_length(clean_words)
        avg_word_length_.append(word_lnth)   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [49]:
final_df = pd.DataFrame(data = [positive_score_, negative_score_, polarity_score_, subjectivity_score_, percent_complex_words_, 
                    avg_sentence_length_, fog_index_, avg_words_per_sentence_, complex_word_count_, word_count_,
                     syllable_count_, personal_pronouns_, avg_word_length_
                    ]).T
final_df.columns = ['positive_score_', 
'negative_score_', 
'polarity_score_', 
'subjectivity_score_', 
'percent_complex_words_', 
'avg_sentence_length_', 
'fog_index_', 
'avg_words_per_sentence_', 
'complex_word_count_', 
'word_count_', 
'syllable_count_', 
'personal_pronouns_', 
'avg_word_length_']

In [50]:
final_df

,positive_score_,negative_score_,polarity_score_,subjectivity_score_,percent_complex_words_,avg_sentence_length_,fog_index_,avg_words_per_sentence_,complex_word_count_,word_count_,syllable_count_,personal_pronouns_,avg_word_length_
0,63.0,33.0,0.007910,0.312501,0.224248,26.697368,10.768647,26.697368,0.0,1199.0,3613.0,2.0,6.984153
1,56.0,38.0,0.011069,0.191490,0.146794,20.862500,8.403718,20.862500,0.0,847.0,2525.0,6.0,6.089728
2,65.0,35.0,0.009012,0.300001,0.208441,22.858824,9.226906,22.858824,0.0,1074.0,3367.0,2.0,6.888268
3,56.0,24.0,0.008426,0.400001,0.142146,19.652174,7.917728,19.652174,0.0,979.0,2773.0,17.0,5.958121
4,48.0,22.0,0.006617,0.371430,0.166496,25.350649,10.206858,25.350649,0.0,1045.0,3106.0,14.0,6.458373
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,22.0,27.0,0.008445,-0.102040,0.190809,20.428571,8.247752,20.428571,0.0,575.0,1683.0,9.0,6.747826
107,36.0,11.0,0.006782,0.531916,0.176471,24.820000,9.998588,24.820000,0.0,698.0,2020.0,3.0,6.540115
108,26.0,44.0,0.009998,-0.257142,0.184723,19.439394,7.849647,19.439394,0.0,709.0,2107.0,2.0,6.486601
109,28.0,3.0,0.008120,0.806453,0.225610,25.230769,10.182552,25.230769,0.0,367.0,1138.0,2.0,6.877384
